#Set up ADK and Gemini API key

In [37]:
!pip -q install --upgrade "google-adk==1.16.0"

import os, sys, subprocess
print("Python:", sys.version)

# 🔑 Put your Gemini API key here (or set GOOGLE_API_KEY in environment)
KEY = os.environ.get("GOOGLE_API_KEY", "").strip() or ""
if KEY == "":
    print("⚠️ Set KEY to your real Gemini API key (AI Studio).")
os.environ["GOOGLE_API_KEY"] = KEY

# Force non-Vertex to avoid any surprises
for var in ["GOOGLE_GENAI_USE_VERTEXAI","GOOGLE_VERTEX_PROJECT","GOOGLE_VERTEX_LOCATION","GOOGLE_CLOUD_PROJECT"]:
    os.environ.pop(var, None)

# Sanity
print(subprocess.run(["adk","--version"], capture_output=True, text=True).stdout.strip())


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
⚠️ Set KEY to your real Gemini API key (AI Studio).
adk, version 1.16.0


#Create the Job Interview Agent package

In [38]:
import os, pathlib, shutil, textwrap

pkg = "job_interview_agent"
shutil.rmtree(pkg, ignore_errors=True)
pathlib.Path(pkg).mkdir(parents=True, exist_ok=True)

open(f"{pkg}/__init__.py","w").write("from .agent import root_agent\n")

# 2a) Question bank
open(f"{pkg}/question_bank.py","w").write(textwrap.dedent("""
from typing import List, Dict

QUESTION_BANK: Dict[str, list[str]] = {
    "data-analyst": [
        "How would you handle missing data and outliers?",
        "Explain the difference between left, right, and inner joins.",
        "When would you choose median over mean? Give an example."
    ],
    "frontend": [
        "How do you structure a React component for reusability?",
        "Explain hydration and server-side rendering.",
        "What’s your approach to accessibility testing?"
    ],
    "generic": [
        "Tell me about the proudest accomplishment in your career.",
        "How do you handle conflict within a team?",
        "Describe a time you made a hard trade-off."
    ]
}

def get_questions(role: str, count: int = 3) -> list[str]:
    role = (role or "").strip().lower()
    bank = QUESTION_BANK.get(role, QUESTION_BANK["generic"])
    count = max(1, min(count, len(bank)))
    return bank[:count]
"""))

# 2b) Tools that return FINAL formatted text
open(f"{pkg}/interview_tools.py","w").write(textwrap.dedent("""
from typing import Tuple
from .question_bank import QUESTION_BANK, get_questions as _get_qs

def get_questions(role: str, count: int = 3) -> str:
    qs = _get_qs(role, count)
    lines = ["Questions:"]
    for i, q in enumerate(qs, start=1):
        lines.append(f"{i}. {q}")
    return "\\n".join(lines)

def _score(answer: str) -> Tuple[int, str]:
    if not answer or len(answer.strip()) < 20:
        return 1, "Answer is too short; add detail and rationale."
    score = 2
    lower = answer.lower()
    if any(k in lower for k in ("because", "why", "tradeoff", "trade-off", "therefore")):
        score += 1
    if any(k in lower for k in ("example", "metric", "latency", "accuracy", "roi", "throughput", "lcp", "cls", "tbt")):
        score += 1
    if len(answer.split()) > 120:
        score = min(score, 4)
    score = max(0, min(score, 5))
    fb = "Clear reasoning with concrete details." if score >= 4 else "Clarify reasoning and add specifics."
    return score, fb

# Make 'question' optional so the tool still works if the LLM omits it.
def score_answer(answer: str, question: str = "") -> str:
    score, fb = _score(answer or "")
    return f"Score: {score}. Feedback: {fb}"
"""))

# 2c) Agent: prints tool output VERBATIM
open(f"{pkg}/agent.py","w").write(textwrap.dedent("""
import os
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from .interview_tools import get_questions, score_answer

# Use the environment key already set in Step 1.
# Ensure Vertex is off
for var in ["GOOGLE_GENAI_USE_VERTEXAI","GOOGLE_VERTEX_PROJECT","GOOGLE_VERTEX_LOCATION","GOOGLE_CLOUD_PROJECT"]:
    os.environ.pop(var, None)

root_agent = Agent(
    model="gemini-2.0-flash",
    name="job_interview_agent",
    description="Generates interview questions and scores answers.",
    instruction=(
        "You have two tools: get_questions(role, count) and score_answer(answer, question?).\\n"
        "ROUTING:\\n"
        "- If the user asks to score/grade an answer, you MUST call score_answer.\\n"
        "- Otherwise, if they ask for interview questions, call get_questions.\\n"
        "OUTPUT:\\n"
        "- Print the tool's return value VERBATIM with NO extra words before/after.\\n"
        "- NEVER output acknowledgements like 'Okay.' or any pre/post text.\\n"
        "FORMATS (already produced by tools):\\n"
        "- Questions:\\n1. ...\\n2. ...\\n"
        "- Score: <0-5>. Feedback: <one sentence>\\n"
    ),
    tools=[FunctionTool(get_questions), FunctionTool(score_answer)],
)
"""))

print("[OK] Scaffolding complete.")
!ls -la {pkg}


[OK] Scaffolding complete.
total 24
drwxr-xr-x 2 root root 4096 Oct 20 06:52 .
drwxr-xr-x 1 root root 4096 Oct 20 06:52 ..
-rw-r--r-- 1 root root 1224 Oct 20 06:52 agent.py
-rw-r--r-- 1 root root   30 Oct 20 06:52 __init__.py
-rw-r--r-- 1 root root 1194 Oct 20 06:52 interview_tools.py
-rw-r--r-- 1 root root  933 Oct 20 06:52 question_bank.py


#Harden agent: tool-only, deterministic

In [41]:
from pathlib import Path

Path("job_interview_agent/agent.py").write_text("""
import os
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from .interview_tools import get_questions, score_answer

# Ensure we never route to Vertex
for var in ["GOOGLE_GENAI_USE_VERTEXAI","GOOGLE_VERTEX_PROJECT","GOOGLE_VERTEX_LOCATION","GOOGLE_CLOUD_PROJECT"]:
    os.environ.pop(var, None)

root_agent = Agent(
    model="gemini-2.0-flash",
    # Make the model deterministic and conservative
    model_params={"temperature": 0.0, "top_p": 0.0},
    name="job_interview_agent",
    description="Generates interview questions and scores answers. Tool-only; prints tool output verbatim.",
    instruction=(
        "TOOL-ONLY MODE. For EVERY user message, you MUST call EXACTLY ONE tool and return its STRING output VERBATIM.\\n"
        "Never type your own prose. Never preface with 'Okay'. Never add extra lines.\\n"
        "\\n"
        "TOOLS AVAILABLE:\\n"
        "- get_questions(role: string, count?: int) -> str  (prints 'Questions:\\n1. ...\\n2. ...')\\n"
        "- score_answer(answer: string, question?: string) -> str  (prints 'Score: <0-5>. Feedback: <...>')\\n"
        "\\n"
        "ROUTING RULES (in priority order):\\n"
        "1) If the message contains a literal call like get_questions(…) or score_answer(…), extract the args and call THAT tool EXACTLY. DO NOT swap tools.\\n"
        "2) Else if the message contains words like 'score' or 'grade' or provides an 'Answer:', call score_answer(answer=?, question=? if present).\\n"
        "3) Otherwise, default to get_questions and infer role/count if needed.\\n"
        "\\n"
        "STRICT OUTPUT: Return the tool's return string EXACTLY. No extra words before or after.\\n"
        "\\n"
        "FEW-SHOT EXAMPLES (DO NOT ECHO):\\n"
        "USER: Call get_questions(role='frontend', count=2) and print verbatim.\\n"
        "ASSISTANT -> (tool:get_questions role=frontend count=2) -> returns 'Questions:\\n1. ...\\n2. ...'\\n"
        "ASSISTANT: Questions:\\n1. ...\\n2. ...\\n"
        "\\n"
        "USER: Call score_answer(answer='SSR...', question='SSR vs CSR?') and print verbatim.\\n"
        "ASSISTANT -> (tool:score_answer answer='SSR...' question='SSR vs CSR?') -> returns 'Score: 4. Feedback: ...'\\n"
        "ASSISTANT: Score: 4. Feedback: ...\\n"
    ),
    tools=[FunctionTool(get_questions), FunctionTool(score_answer)],
)
""")
print("[OK] agent hardened (tool-only, temp=0)")


[OK] agent hardened (tool-only, temp=0)


#Finalize agent config (remove model params)

In [43]:
from pathlib import Path

Path("job_interview_agent/agent.py").write_text("""
import os
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from .interview_tools import get_questions, score_answer

# Keep requests on Gemini API (not Vertex) if any envs are present
for var in ["GOOGLE_GENAI_USE_VERTEXAI","GOOGLE_VERTEX_PROJECT","GOOGLE_VERTEX_LOCATION","GOOGLE_CLOUD_PROJECT"]:
    os.environ.pop(var, None)

root_agent = Agent(
    model="gemini-2.0-flash",
    name="job_interview_agent",
    description="Generates interview questions and scores answers. Tool-only; prints tool output verbatim.",
    instruction=(
        "TOOL-ONLY MODE. For EVERY user message, you MUST call EXACTLY ONE tool and return its STRING output VERBATIM.\\n"
        "Never type your own prose. Never add extra words.\\n"
        "\\n"
        "TOOLS:\\n"
        "- get_questions(role: string, count?: int) -> str  (returns 'Questions:\\n1. ...\\n2. ...')\\n"
        "- score_answer(answer: string, question?: string) -> str  (returns 'Score: <0-5>. Feedback: <...>')\\n"
        "\\n"
        "ROUTING (priority):\\n"
        "1) If the user gives a literal call like get_questions(...) or score_answer(...), extract those args and call THAT tool.\\n"
        "2) Else if the message says 'score'/'grade' or includes an 'Answer:', call score_answer(answer=?, question=? if present).\\n"
        "3) Otherwise, default to get_questions and infer role/count if needed.\\n"
        "\\n"
        "STRICT OUTPUT: Return the tool's return string EXACTLY.\\n"
    ),
    tools=[FunctionTool(get_questions), FunctionTool(score_answer)],
)
""")
print("[OK] Removed model_params; agent hardened")


[OK] Removed model_params; agent hardened


#CLI sanity checks (generate & score)

In [44]:
import subprocess, re, os

def run_adk_once(agent_pkg: str, user_message: str) -> str:
    payload = user_message.strip() + "\nexit\n"
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=payload.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        check=False,
        env=dict(os.environ),
    )
    out = p.stdout.decode("utf-8", "replace")
    # Keep just the agent’s reply
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*?)(?:\n\[user\]:|\Z)", out, re.S)
    return (m.group(1).strip() if m else out).strip()

print(">>> Q generation")
print(run_adk_once(
    "job_interview_agent",
    "Call get_questions(role='frontend', count=2) and print the tool output verbatim."
))

print("\n>>> Scoring")
print(run_adk_once(
    "job_interview_agent",
    "Call score_answer(answer='SSR renders on the server; hydration attaches event handlers on the client. "
    "I compare LCP/TBT/CLS and conversion impact, choosing based on SEO & latency.') and print the tool output verbatim."
))


>>> Q generation
Questions:
1. How do you structure a React component for reusability?
2. Explain hydration and server-side rendering.

>>> Scoring
Score: 3. Feedback: Clarify reasoning and add specifics.


#Automated smoke tests

In [45]:
# tests_smoke.py
import re, os, subprocess

def run_once(agent_pkg: str, msg: str) -> str:
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(msg.strip()+"\nexit\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False, env=os.environ,
    )
    out = p.stdout.decode("utf-8","replace")
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*?)(?:\n\[user\]:|\Z)", out, re.S)
    return (m.group(1).strip() if m else out).strip()

def expect(txt, patterns):
    missing = [p for p in patterns if not re.search(p, txt, re.S)]
    if missing:
        raise AssertionError(f"Missing: {missing}\n-- Got --\n{txt}")

# Tests
qs = run_once("job_interview_agent",
              "Call get_questions(role='frontend', count=2) and print the tool output verbatim.")
expect(qs, [r"^Questions:\s*\n1\.", r"\n2\."])

sc = run_once("job_interview_agent",
              "Call score_answer(answer='SSR renders on the server; hydration attaches client handlers. "
              "I track LCP/TBT/CLS and pick based on SEO/latency.') and print the tool output verbatim.")
expect(sc, [r"^Score:\s*[0-5]\b", r"Feedback:\s*.+"])

print("Smoke tests passed ✅")


Smoke tests passed ✅


#Demo runner: save sample outputs

In [51]:
%%bash
cat > demo_runner.py <<'PY'
import os, re, subprocess, datetime

def run_once(agent_pkg, msg):
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(msg.strip()+"\nexit\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False, env=os.environ
    )
    out = p.stdout.decode("utf-8","replace")
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*?)(?:\n\[user\]:|\Z)", out, re.S)
    return (m.group(1).strip() if m else out).strip()

q_prompt = "Call get_questions(role='data-analyst', count=3) and print the tool output verbatim."
s_prompt = ("Call score_answer(answer='Normalization reduces redundancy; denormalization speeds reads. "
            "I use 3NF for OLTP, denormalize for OLAP; validate with query profiling.') "
            "and print the tool output verbatim.")

qs = run_once("job_interview_agent", q_prompt)
sc = run_once("job_interview_agent", s_prompt)

ts = datetime.datetime.now().isoformat(timespec="seconds")
with open("demo_output.txt","w") as f:
    f.write(f"=== Demo run @ {ts} ===\n\n")
    f.write("Prompt 1 (Questions):\n" + q_prompt + "\n\n")
    f.write("Output 1:\n" + qs + "\n\n")
    f.write("Prompt 2 (Scoring):\n" + s_prompt + "\n\n")
    f.write("Output 2:\n" + sc + "\n")

print("Wrote demo_output.txt")
PY

python demo_runner.py
sed -n '1,200p' demo_output.txt


Wrote demo_output.txt
=== Demo run @ 2025-10-20T07:34:57 ===

Prompt 1 (Questions):
Call get_questions(role='data-analyst', count=3) and print the tool output verbatim.

Output 1:
Questions:
1. How would you handle missing data and outliers?
2. Explain the difference between left, right, and inner joins.
3. When would you choose median over mean? Give an example.

Prompt 2 (Scoring):
Call score_answer(answer='Normalization reduces redundancy; denormalization speeds reads. I use 3NF for OLTP, denormalize for OLAP; validate with query profiling.') and print the tool output verbatim.

Output 2:
Score: 2. Feedback: Clarify reasoning and add specifics.


#Standalone test script: write & run

In [53]:
%%bash
cat > tests_smoke.py << 'PY'
import re, os, subprocess

def run_once(agent_pkg: str, msg: str) -> str:
    p = subprocess.run(
        ["adk","run",agent_pkg],
        input=(msg.strip()+"\nexit\n").encode(),
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=False, env=os.environ,
    )
    out = p.stdout.decode("utf-8","replace")
    m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*?)(?:\n\[user\]:|\Z)", out, re.S)
    return (m.group(1).strip() if m else out).strip()

def expect(txt, patterns):
    missing = [p for p in patterns if not re.search(p, txt, re.S)]
    if missing:
        raise AssertionError(f"Missing: {missing}\n-- Got --\n{txt}")

qs = run_once("job_interview_agent",
              "Call get_questions(role='frontend', count=2) and print the tool output verbatim.")
expect(qs, [r"^Questions:\s*\n1\.", r"\n2\."])

sc = run_once("job_interview_agent",
              "Call score_answer(answer='SSR renders on the server; hydration attaches client handlers. "
              "I track LCP/TBT/CLS and pick based on SEO/latency.') and print the tool output verbatim.")
expect(sc, [r"^Score:\s*[0-5]\b", r"Feedback:\s*.+"])

print("Smoke tests passed ✅")
PY

python tests_smoke.py


Smoke tests passed ✅


#Pin websockets and install Gradio

In [55]:
# 1) Pin websockets to satisfy both ADK and google-genai
!pip -q install --upgrade "websockets==15.0.1"

# 2) (Re)install gradio but keep our pinned websockets
!pip -q install --upgrade "gradio==5.4.0"

# 3) Make sure Python actually uses the new websockets module in this session
import sys
for m in [m for m in list(sys.modules) if m.startswith("websockets")]:
    del sys.modules[m]

import websockets
print("websockets version now:", websockets.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio-client 1.4.2 requires websockets<13.0,>=10.0, but you have websockets 15.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.16.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
google-genai 1.44.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
websockets version now: 12.0


#Gradio UI for the Interview Agent

In [ ]:
# --- Robust Gradio UI that shells out to ADK CLI (no async) ---

import gradio as gr
import subprocess, re, os

AGENT = "job_interview_agent"

def _run_adk_once(agent_pkg: str, user_message: str) -> str:
    try:
        payload = user_message.strip() + "\nexit\n"
        p = subprocess.run(
            ["adk", "run", agent_pkg],
            input=payload.encode(),
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            check=False,
            env=dict(os.environ),
        )
        out = p.stdout.decode("utf-8", "replace")
        m = re.search(rf"\[{re.escape(agent_pkg)}\]:\s*(.*?)(?:\n\[user\]:|\Z)", out, re.S)
        return (m.group(1).strip() if m else out).strip()
    except Exception as e:
        return f"(error) {type(e).__name__}: {e}"

def ask_questions(role: str, count: int) -> str:
    msg = f"Call get_questions(role='{role}', count={int(count)}) and print the tool output verbatim."
    return _run_adk_once(AGENT, msg)

def score_answer(question: str, answer: str) -> str:
    answer = (answer or "").strip()
    if not answer:
        return "Score: 1. Feedback: Answer is too short; add detail and rationale."
    if question and question.strip():
        msg = f"Call score_answer(answer='{answer}', question='{question}') and print the tool output verbatim."
    else:
        msg = f"Call score_answer(answer='{answer}') and print the tool output verbatim."
    return _run_adk_once(AGENT, msg)

with gr.Blocks(title="Job Interview Agent (ADK)") as demo:
    gr.Markdown("## Job Interview Agent · ADK + Gemini\nGenerate interview questions and score answers using your agent tools.")

    with gr.Tab("Generate Questions"):
        with gr.Row():
            role = gr.Dropdown(choices=["frontend","data-analyst","generic"], value="frontend", label="Role")
            count = gr.Slider(1, 5, value=2, step=1, label="Number of questions")
        out_q = gr.Textbox(label="Questions", lines=8)
        gr.Button("Generate").click(ask_questions, inputs=[role, count], outputs=out_q)

    with gr.Tab("Score Answer"):
        q = gr.Textbox(label="Question (optional)", placeholder="e.g., Explain hydration and SSR.")
        a = gr.Textbox(label="Your Answer", lines=6, placeholder="Paste or type your answer here…")
        out_s = gr.Textbox(label="Score & Feedback", lines=2)
        gr.Button("Score").click(score_answer, inputs=[q, a], outputs=out_s)

# In Colab it's safer to enable share + debug to see errors inline if any
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://069bab35f4a719d7c2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
